In [ ]:
%matplotlib inline
import os
from os import walk
from os.path import join as pjoin
import matplotlib.pyplot as plt
import geopandas as gpd
from Utilities import track
from shapely.geometry import LineString, Point
from shapely.geometry import box as sbox
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')

def filter_tracks_domain(df, minlon=90, maxlon=180, minlat=-40, maxlat=0):
    """
    Takes a `DataFrame` and filters on the basis of whether the track interscts
    the given domain, which is specified by the minimum and maximum longitude and 
    latitude.
    
    NOTE: This assumes the tracks and bounding box are in the same geographic 
    coordinate system (i.e. generally a latitude-longitude coordinate system). 
    It will NOT support different projections (e.g. UTM data for the bounds and
    geographic for the tracks).
    
    NOTE: This doesn't work if there is only one point for the track. 
    
    :param df: :class:`pandas.DataFrame` that holds the TCLV data
    :param float minlon: minimum longitude of the bounding box
    :param float minlat: minimum latitude of the bounding box
    :param float maxlon: maximum longitude of the bounding box
    :param float maxlat: maximum latitude of the bounding box
    """
    domain = sbox(minlon, minlat, maxlon, maxlat, ccw=False)
    tracks = df.groupby('num')
    tempfilter = tracks.filter(lambda x: len(x) > 1)
    tempfilter.head()
    filterdf = tempfilter.groupby('num').filter(lambda x: LineString(zip(x['lon'], x['lat'])).intersects(domain))
    return filterdf

def isLeft(line, point):
    """
    Test whether a point is to the left of a (directed) line segment. 
    
    :param line: :class:`Shapely.geometry.LineString` of the line feature being tested
    :param point: :class:`Shapely.geometry.Point` being tested
    """
    start = Point(line.coords[0])
    end = Point(line.coords[1])

    det = (end.x - start.x) * (point.y - start.y) - (end.y - start.y) * (point.x - start.x)

    if det > 0: return True
    if det <= 0: return False

def isLandfall(gate, tracks):
    crossings = tracks.crosses(gate.geometry)
    landfall = []
    for t in tracks[crossings].itertuples():
        if isLeft(gate.geometry, Point(t.geometry.coords[0])):
            landfall.append(True)
        else:
            landfall.append(False)

    return tracks[crossings][landfall]

def countCrossings(gates, tracks, sim):
    gates['sim'] = sim
    for i, gate in enumerate(gates.itertuples(index=False)):
        ncrossings = 0
        l = isLandfall(gate, tracks)
        ncrossings = len(l)
        #crossings = tracks.crosses(gate.geometry)
        #ncrossings = np.sum(tracks.crosses(gate.geometry))
        if ncrossings > 0:
            gates['count'].iloc[i] = ncrossings
            gates['meanlfintensity'].iloc[i] = l['pmin'].mean()
            gates['minlfintensity'].iloc[i] = l['pmin'].min()
            cathist, bins = np.histogram(l['category'].values, bins=[0,1,2,3,4,5, 6])
            gates['cat1'].iloc[i] = cathist[1]
            gates['cat2'].iloc[i] = cathist[2]
            gates['cat3'].iloc[i] = cathist[3]
            gates['cat4'].iloc[i] = cathist[4]
            gates['cat5'].iloc[i] = cathist[5]
        else:
            gates['meanlfintensity'].iloc[i] = np.nan
            gates['minlfintensity'].iloc[i] = np.nan
            gates['cat1'].iloc[i] = 0
            gates['cat2'].iloc[i] = 0
            gates['cat3'].iloc[i] = 0
            gates['cat4'].iloc[i] = 0
            gates['cat5'].iloc[i] = 0
    return gates


Start with reading in the gates into a GeoDataFrame, and adding some additional attributes. This GeoDataFrame will be duplicated for each simulation, then aggregated for the summary statistics.

In [ ]:
gates = gpd.read_file("C:/WorkSpace/data/tcha/gates.shp")
gates['sim'] = 0
gates['count'] = 0
gates['meanlfintensity'] = np.nan
gates['minlfintensity'] = np.nan
gates['cat1'] = 0
gates['cat2'] = 0
gates['cat3'] = 0
gates['cat4'] = 0
gates['cat5'] = 0

In [ ]:
obstc = pd.read_csv('../ibtracs.since1980.list.v04r00.csv', 
                       skiprows=[1],
                       usecols=[0,6,8,9,11,113],
                       na_values=[' '],
                       parse_dates=[1])
obstc.rename(columns={'SID':'num', 'LAT': 'lat', 'LON':'lon', 'WMO_PRES':'pmin', 'BOM_POCI':'poci'}, inplace=True)
obstc = filter_tracks_domain(obstc)
trackgdf = []
for k, t in obstc.groupby('num'):
    segments = []
    for n in range(len(t.num) - 1):
        segment = LineString([[t.lon.iloc[n], t.lat.iloc[n]],[t.lon.iloc[n+1], t.lat.iloc[n+1]]])
        segments.append(segment)

    gdf = gpd.GeoDataFrame.from_records(t[:-1])
    gdf['geometry'] = segments
    gdf['category'] = pd.cut(gdf['pmin'], 
                             bins=[0, 930, 955, 970, 985, 990, 1020], 
                             labels=[5,4,3,2,1,0])
    trackgdf.append(gdf)

trackgdf = pd.concat(trackgdf)

In [ ]:
gatedf = gates.copy()
gatedf = countCrossings(gatedf, trackgdf, 0)

In [ ]:
gatedf

In [ ]:
width=0.4
fig, ax = plt.subplots(3,1,figsize=(12,16),sharex=True)
cat12 = np.add(gates['cat1'], gatedf['cat2']).tolist()
cat123 = np.add(cat12, gatedf['cat3']).tolist()
cat1234 = np.add(cat123, gatedf['cat4']).tolist()
ax[0].bar(gatedf['gate'],gatedf['cat1'], color='b', label="Cat 1")
ax[0].bar(gatedf['gate'], gatedf['cat2'], bottom=gatedf['cat1'], color='g', label='Cat 2')
ax[0].bar(gatedf['gate'], gatedf['cat3'], bottom=cat12, color='y', label='Cat 3')
ax[0].bar(gatedf['gate'], gatedf['cat4'], bottom=cat123, color='orange', label='Cat 4')
ax[0].bar(gatedf['gate'], gatedf['cat5'], bottom=cat1234, color='r', label='Cat 5')

ax[0].legend()
ax[0].set_ylabel("Number of TCs")
ax[1].plot(gatedf['gate'], gatedf['minlfintensity'], label='Minimum landfall intensity')
ax[1].plot(gatedf['gate'], gatedf['meanlfintensity'], color='r', label='Mean landfall intensity')
#ax[1].fill_between(gates['gate'], gates['meanlfintensity_q10'],
#                   gates['meanlfintensity_q90'],color='r', alpha=0.25)

ax[1].legend(loc=2)
ax[1].set_ylim((850, 1020))
ax[1].set_ylabel("Pressure (hPa)")
ax[2].bar(gatedf['gate'], gatedf['count']/40.)
ax[2].set_xlim((0,48))
ax[2].set_xticks(np.arange(0,48,2))
ax[2].set_yticks(np.arange(0,0.51,.05))
ax[2].set_xticklabels(gatedf['label'][::2], rotation='vertical')
ax[2].set_ylabel("Mean rate of landfall")